In [17]:
small_tile

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15]])

In [25]:
np.arange(10000)[arr]

array([[ 0,  1,  2,  9, 10, 11, 18, 19, 20],
       [ 3,  4,  5, 12, 13, 14, 21, 22, 23],
       [ 6,  7,  8, 15, 16, 17, 24, 25, 26],
       [27, 28, 29, 36, 37, 38, 45, 46, 47],
       [30, 31, 32, 39, 40, 41, 48, 49, 50],
       [33, 34, 35, 42, 43, 44, 51, 52, 53],
       [54, 55, 56, 63, 64, 65, 72, 73, 74],
       [57, 58, 59, 66, 67, 68, 75, 76, 77],
       [60, 61, 62, 69, 70, 71, 78, 79, 80]])

In [1]:
import numpy as np

ten_thousand = np.arange(10000)

edge_sqrt = 3
small_tile_array = np.arange(edge_sqrt ** 2)
small_tile = np.reshape(small_tile_array, (edge_sqrt, edge_sqrt))
a = np.repeat(small_tile, edge_sqrt, axis=0)
b = np.repeat(a, edge_sqrt, axis=1)

tiled = np.tile(small_tile, (edge_sqrt, edge_sqrt))
arr = tiled + b * edge_sqrt ** 2

ten_thousand[arr]


array([[ 0,  1,  2,  9, 10, 11, 18, 19, 20],
       [ 3,  4,  5, 12, 13, 14, 21, 22, 23],
       [ 6,  7,  8, 15, 16, 17, 24, 25, 26],
       [27, 28, 29, 36, 37, 38, 45, 46, 47],
       [30, 31, 32, 39, 40, 41, 48, 49, 50],
       [33, 34, 35, 42, 43, 44, 51, 52, 53],
       [54, 55, 56, 63, 64, 65, 72, 73, 74],
       [57, 58, 59, 66, 67, 68, 75, 76, 77],
       [60, 61, 62, 69, 70, 71, 78, 79, 80]])

In [4]:
from transformers import ViTMAEConfig, ViTMAEModel, ViTMAEForPreTraining
%load_ext autoreload


# Initializing a ViT MAE vit-mae-base style co`nfiguration
configuration = ViTMAEConfig(
    image_size=100,
    num_channels=1,
    hidden_size=480,
    intermediate_size=1024,
    decoder_intermediate_size=1024,
    patch_size=10,
    mask_ratio=0,
)

# Initializing a model (with random weights) from the vit-mae-base style configuration
model = ViTMAEForPreTraining(configuration)

# Accessing the model configuration
configuration = model.config

print('number of parameters: ', sum(p.numel() for p in model.parameters()))


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
number of parameters:  40191300


In [5]:
from dataloader import BATCH_SIZE, square_xrd_dataloader, square_binary_dataloader

import torch
from torch import nn, optim
mse_loss = nn.MSELoss()

def train_model(num_epochs=100):
    outputs = []
    optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-5)
    for epoch in range(num_epochs):
        for idx, data in enumerate(square_binary_dataloader):
            # ===================forward=====================
            output = model(data)
            loss = mse_loss(output.logits, data.squeeze())
            # ===================backward====================
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()


            if idx % 5 == 0:
                print(f"Finished batch {idx} in epoch {epoch + 1}. Loss: {loss.item():.4f}")

        print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1, num_epochs, loss.item()))
        outputs.append((epoch, data, output))



# Train the model

model.train(True)
train_model(num_epochs=1)
model.train(False)

Finished batch 0 in epoch 1. Loss: 1.1127
Finished batch 5 in epoch 1. Loss: 26.1782
Finished batch 10 in epoch 1. Loss: 2.4455


KeyboardInterrupt: 

In [13]:
# this is a demo for testing

from transformers import AutoFeatureExtractor, ViTMAEForPreTraining
from PIL import Image
import requests

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/vit-mae-base")
model_pretrained = ViTMAEForPreTraining.from_pretrained("facebook/vit-mae-base")

inputs = feature_extractor(images=image, return_tensors="pt")
outputs = model_pretrained(**inputs)
loss = outputs.loss
mask = outputs.mask
ids_restore = outputs.ids_restore

inputs['pixel_values'].type()